# Multiple linear regression

## Import the relevant libraries

In [249]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression
import scipy.stats as stat

sns.set()

class LinearRegression(LinearRegression):
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                 n_jobs=1):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.copy_X = copy_X
        self.n_jobs = n_jobs

    
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)
        
        # Calculate SSE (sum of squared errors)
        # and SE (standard error)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])

        # compute the t-statistic for each feature
        self.t = self.coef_ / se
        # find the p-value for each feature
        self.p = np.squeeze(2 * (1 - stat.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1])))
        return self

## Load the data

In [250]:
data = pd.read_csv('1.02.csv')
data.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [251]:
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


## Train/test split

In [252]:
data, data_test = train_test_split(data, test_size=0.2, random_state=42)
data.shape, data_test.shape

((67, 3), (17, 3))

## Create the multiple linear regression

### Declare the dependent and independent variables

In [253]:
x = data[['SAT','Rand 1,2,3']]
y = data['GPA']

## Standardization

In [254]:
scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

### Regression itself

In [255]:
reg = LinearRegression()
reg.fit(x_scaled,y)

LinearRegression()

In [256]:
reg.coef_

array([ 0.15401858, -0.02813112])

In [257]:
reg.intercept_

3.341044776119403

## R-Squared

In [258]:
reg.score(x_scaled,y)

0.3806985325264898

### Adjusted Formula

R2adj = 1 - (1-R^2)*((n-1)/(n-p-1))

In [259]:
r2adj = 1-(1-reg.score(x_scaled,y))*((x_scaled.shape[0]-1)/(x_scaled.shape[0]-x_scaled.shape[1]-1))
r2adj

0.3613453616679426

## Feature Selection with F-regression

In [260]:
f_regression(x_scaled,y)

(array([37.96161426,  1.38986256]), array([5.10187427e-08, 2.42725838e-01]))

In [261]:
p_values = f_regression(x_scaled,y)[1]
p_values

array([5.10187427e-08, 2.42725838e-01])

In [262]:
p_values.round(3)

array([0.   , 0.243])

## Using custom class

In [263]:
#p-value
reg.p

array([5.44646848e-08, 2.65842582e-01])

In [264]:
#t-statistic
reg.t

array([[ 6.14489232, -1.12234971]])

## Summary table

In [265]:
reg_summary = pd.DataFrame(data=np.concatenate([np.array(["Bias"]), x.columns.values]), columns=['Features'])
reg_summary

,Features
0,Bias
1,SAT
2,"Rand 1,2,3"


In [266]:
reg_summary['Coefficients/Weights'] = np.concatenate([np.array([reg.intercept_]), reg.coef_])
reg_summary['P-value'] = np.concatenate([np.array(["N/A"]), reg.p.round(4)])
reg_summary

,Features,Coefficients/Weights,P-value
0,Bias,3.341045,N/A
1,SAT,0.154019,0.0
2,"Rand 1,2,3",-0.028131,0.2658


## Predictions

In [267]:
new_data = pd.DataFrame(data=[[1700,2],[1800,1]], columns=['SAT', 'Rand 1,2,3'])
scaled_new_data = scaler.transform(new_data)
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [268]:
reg.predict(scaled_new_data)

array([3.12593611, 3.30451806])

## Removing useless variables

In [269]:
reg_simple = LinearRegression()
x_simple_matrix = x_scaled[:,0].reshape(-1,1)
reg_simple.fit(x_simple_matrix,y)

LinearRegression()

In [270]:
reg_simple.predict(scaled_new_data[:,0].reshape(-1,1))

array([3.12264734, 3.26951704])